In [ ]:
import csv

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
competitors = pd.read_csv('RheinlandPfalzOpen2022-registration-10-07-2022.csv')

In [ ]:
competitors

In [ ]:
n_all_competitors = len(competitors)

In [ ]:
competitors.keys()

In [ ]:
results = pd.read_csv('../../WCA/WCA_export.tsv/WCA_export_Results.tsv', sep='\t')

In [ ]:
results

In [ ]:
results.keys()

# Data prep
extract relevant information

In [ ]:
events_daily = {
                'sat' : ['555', '666', '777', '333bf', '333oh', 'minx', 'skewb', 'sq1'],
                'sun' : ['333', '222', '444', 'clock', 'pyram']
                }

<hr>

In [ ]:
# count competitors + guests where >=1 event of competitor is held on saturday or sunday, respectively
is_attending_sat = competitors[events_daily['sat']].any(1)
is_attending_sun = competitors[events_daily['sun']].any(1)
is_attending_both = is_attending_sat & is_attending_sun
n_comp_sat = is_attending_sat.sum()
n_comp_sun = is_attending_sun.sum()
n_comp_both = is_attending_both.sum()
attendees_sat = competitors.loc[is_attending_sat]
attendees_sun = competitors.loc[is_attending_sun]

<hr>

In [ ]:
n_guests_sat = attendees_sat['Guests'].sum()
n_guests_sun = attendees_sun['Guests'].sum()

<hr>

In [ ]:
is_newcomer = competitors['WCA ID'].isnull()
newcomers = competitors.loc[is_newcomer]
newcomers_only_sat = is_newcomer & is_attending_sat & ~is_attending_sun
newcomers_only_sun = is_newcomer & ~is_attending_sat & is_attending_sun
newcomers_both_days = is_newcomer & is_attending_sat & is_attending_sun
n_newcomers = is_newcomer.sum()
n_newcomers_sat = newcomers_only_sat.sum()
n_newcomers_sun = newcomers_only_sun.sum()
n_newcomers_both = newcomers_both_days.sum()

<hr>

In [ ]:
is_foreigner = competitors['Country'] != 'Germany'
foreigners_only_sat = is_foreigner & is_attending_sat & ~is_attending_sun
foreigners_only_sun = is_foreigner & ~is_attending_sat & is_attending_sun
foreigners_both_days = is_foreigner & is_attending_sat & is_attending_sun
n_foreigners = is_foreigner.sum()
n_foreigners_sat = foreigners_only_sat.sum()
n_foreigners_sun = foreigners_only_sun.sum()
n_foreigners_both = foreigners_both_days.sum()

In [ ]:
# necessary to decide whether to also hold an English tutorial besides a German one
is_foreigner_and_newcomer = is_foreigner & is_newcomer
foreigners_newcomers_only_sat = foreigners_only_sat & newcomers_only_sat
foreigners_newcomers_only_sun = foreigners_only_sun & newcomers_only_sun
foreigners_newcomers_both_days = foreigners_both_days & newcomers_both_days
n_foreign_new = is_foreigner_and_newcomer.sum()
n_foreign_new_sat = foreigners_newcomers_only_sat.sum()
n_foreign_new_sun = foreigners_newcomers_only_sun.sum()
n_foreign_new_both = foreigners_newcomers_both_days.sum()

<hr>

In [ ]:
# number of competitions is useful to know if someone does not yet have that much experience,
# e.g. ""SELECT count(distinct competitionId) FROM Results where personId='$chosenPersonId'""
# as in https://github.com/thewca/worldcubeassociation.org/commit/1e41fd736f2efea9258a1a954d096359a472ade5
is_returner = ~competitors['WCA ID'].isnull()
returners = competitors.loc[is_returner]

In [ ]:
result_is_from_returner = results.personId.isin(returners['WCA ID'])

In [ ]:
# this is great: https://stackoverflow.com/a/38309823
n_previous_comps = results[result_is_from_returner].groupby('personId')['competitionId'].nunique()

In [ ]:
# boolean mask, for every returner
has_only_few_comps = n_previous_comps <= 2

In [ ]:
returners_with_few_comps = returners.loc[returners['WCA ID'].isin(has_only_few_comps[has_only_few_comps].index)]

In [ ]:
inexperienced_returner_is_attending_sat = returners_with_few_comps[events_daily['sat']].any(1)
inexperienced_returner_is_attending_sun = returners_with_few_comps[events_daily['sun']].any(1)
inexperienced_returner_only_sat = inexperienced_returner_is_attending_sat & ~inexperienced_returner_is_attending_sun
inexperienced_returner_only_sun = ~inexperienced_returner_is_attending_sat & inexperienced_returner_is_attending_sun
inexperienced_returner_both_days = inexperienced_returner_is_attending_sat & inexperienced_returner_is_attending_sun
n_inexperienced = has_only_few_comps.sum()
n_inexperienced_sat = inexperienced_returner_only_sat.sum()
n_inexperienced_sun = inexperienced_returner_only_sun.sum()
n_inexperienced_both = inexperienced_returner_both_days.sum()

# Function for convenience
for some results, I really want to see all the entries, it's not too bad here

In [ ]:
# from https://stackoverflow.com/a/19126566
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

# Daily attendance
plan number of chairs, tables accordingly

this is independent of attendee's attributes, just to know the venue occupancy, especially in covid times

In [ ]:
print('\t\t',
      'Saturday\t',
      'Sunday')
print('Competitors\t',
      n_comp_sat,
      '\t\t',
      n_comp_sun)
print('Guests\t',
      '\t',
      n_guests_sat,
      '\t\t',
      n_guests_sun)
print('Total\t',
      '\t',
      n_comp_sat+n_guests_sat,
      '\t\t',
      n_comp_sun+n_guests_sun)

# Returners
people with a WCA ID

In [ ]:
print_full(returners['WCA ID'])

In [ ]:
print_full(n_previous_comps)

In [ ]:
# where did they go?
results.loc[result_is_from_returner]['competitionId'].unique()

# Newcomers and less experienced competitors
for tutorial, can be helpful to know how many people we expect to join

In [ ]:
newcomers

In [ ]:
returners_with_few_comps

In [ ]:
print('\t\t\t',
      'Only Saturday\t',
      'Only Sunday\t',
      'Both days\t',
      'Any day')
print('Newcomers\t\t',
      n_newcomers_sat,
      '\t','\t',
      n_newcomers_sun,
      '\t','\t',
      n_newcomers_both,
      '\t','\t',
      n_newcomers)
print('Fraction\t\t',
      round(n_newcomers_sat/n_comp_sat,2),
      '\t','\t',
      round(n_newcomers_sun/n_comp_sun,2),
      '\t','\t',
      round(n_newcomers_both/n_comp_both,2),
      '\t','\t',
      round(n_newcomers/n_all_competitors,2))
print()
print('Foreign newcomers\t',
      n_foreign_new_sat,
      '\t','\t',
      n_foreign_new_sun,
      '\t','\t',
      n_foreign_new_both,
      '\t','\t',
      n_foreign_new)
print('Fraction\t\t',
      round(n_foreign_new_sat/n_comp_sat,2),
      '\t','\t',
      round(n_foreign_new_sun/n_comp_sun,2),
      '\t','\t',
      round(n_foreign_new_both/n_comp_both,2),
      '\t','\t',
      round(n_foreign_new/n_all_competitors,2))
print()
print('Returners (<= 2 comps)\t',
      n_inexperienced_sat,
      '\t','\t',
      n_inexperienced_sun,
      '\t','\t',
      n_inexperienced_both,
      '\t','\t',
      n_inexperienced)
print('Fraction\t\t',
      round(n_inexperienced_sat/n_comp_sat,2),
      '\t','\t',
      round(n_inexperienced_sun/n_comp_sun,2),
      '\t','\t',
      round(n_inexperienced_both/n_comp_both,2),
      '\t','\t',
      round(n_inexperienced/n_all_competitors, 2))

# Demography
basic stats of who is attending

In [ ]:
gender = competitors.Gender

In [ ]:
gender_vals = gender.value_counts()

In [ ]:
gender_vals

In [ ]:
gender_vals.plot(kind='bar')

In [ ]:
print_full(competitors[['Name','WCA ID','Birth Date']].sort_values(by=['Birth Date']))